# Implementing elementary cellular automaton in Q#

**Dmytro Fedoriaka, December 12, 2021.**

*This notebook is part of the [Q# Advent Calendar 2021](https://devblogs.microsoft.com/qsharp/q-advent-calendar-2021/). Check it out for more great posts about quantum computing and Q#!*

Hello!

Today we are going to implement an [Elementary Cellular Automaton](https://en.wikipedia.org/wiki/Elementary_cellular_automaton) on a quantum computer using [Q# programming language](https://en.wikipedia.org/wiki/Q_Sharp).

## 1. What is Cellular Automaton?

[Cellular Automata](https://en.wikipedia.org/wiki/Cellular_automaton) (CA) are models of computations introduced by John von Neumann 1960s. They are made of "cells" which can be in one of two states, and rules how these cells change based on state of their neigbors. Interesting thing about them is that even for very simple rules they can show surprisngly complex behaviour. One famous example of CA is John Conway's [Game of Life](https://en.wikipedia.org/wiki/Conway%27s_Game_of_Life).

Recently I asked myself a question: can we simulate cellular automata on a quantum computer? I read some papers and turns out there is a lot of research on [Quantum Cellular Automata](https://en.wikipedia.org/wiki/Quantum_cellular_automaton) - quantum systems which in some ways resemble usual CA. But that was not quite what I wanted. I wanted to replace cells with qubits and rules with unitary evolution. Turned out this is generally not posible, because quantum computation must be reversible. 

So, I considered the simplest class of CA - Elementary Cellular Automata (ECA). They are 1-dimensional (all celss are in a row, or an array). Rule defines state of next cell based on state of ti and two its neigbors on previous step. They were introduced by Stephen Wolfram in 1980s. There are 256 different rules for Elementary Cellular Automata, and some of them can be simulated on quantum computer.

In this notebook I am going to focus on one particular ECA - so-called "Rule 90".

## 2. Rule 90

Rule 90 is one particular ECA. It has its own page on [Wikipedia](https://en.wikipedia.org/wiki/Rule_90) and [Wolfram Mathwrold](https://mathworld.wolfram.com/Rule90.html).

The picture below (from MathWorld) descibes this rule:

![image](https://mathworld.wolfram.com/images/eps-gif/ElementaryCARule090_1000.gif)

Eight pictures on the top describe what is state of cell on next step based on previous step. Here white cell means "0" and black cell means "1". If you look carefully at them you will realize that this rule replaces the cell with XOR of its left and right neigbour - we will use this observation below to build quantum circuit for this ECA. Also note that if you read digits in top row as binary number, you will get 90 - that's why it's called "Rule 90".

Picture on the bottom show evolution of single black cell. Every row is a step or "generation", and time flows from top tp bottom. 

Before we move on, we have to make two assumtions (so we can implement this rule with quantum circuit):

* Number of cells in a row, $N$, is an even number.
* Border condition: we assume that there is cell with "0" in it to the left of leftmost cell and to the right of rightmost cell.

## 3. Quantum circuit for Rule 90

So, we want to implement a quantum circuit which would act on basis states as Rule 90 acts on clasical bit strings. For now we don't care about superpositions, so we can think about qubits as of classical bits.

The hardest part of our task is that we need to do everything in-place: input and output are the same qubits, we don't have extra ("ancilla") qubits.

First useful gate for us is the [CNOT](https://en.wikipedia.org/wiki/Controlled_NOT_gate) gate. 
CNOT($q_1$, $q_2$) acts like X(q2) if $q_1$ is in state $|1 \rangle$, and does nothing if $q_1$ is in state $|0 \rangle$. So, it replaces $q_2$ with $q_1 \oplus q_2$.

Rule 90 XORs every cell with another cell at distance 2. Let's try to do the same: let's apply 
CNOT($q_{N-3}, q_{N-1}$), CNOT($q_{N-4}, q_{N-2}$), ...,  CNOT($q_{0}, q_{2}$).

This will get us the following result (classically):

$q_{N-1} \leftarrow q_{N-1} \oplus q_{N-3}$

$q_{N-2} \leftarrow q_{N-2} \oplus q_{N-4}$

...

$q_{2} \leftarrow q_{2} \oplus q_{0}$

This is almost what we want, except everything is shifted 1 "cell" to the right, and we did nothoing to $q_1$ and $q_0$.





![image](pic/rule90-circuit.png)

## 4. The Q# code

In the code below:

* Operation "Rule90" applies Rule 90 to given register of qubits.
* Operation "MeasureAndPrint" measures register (in computational basis) and prints result.
* Operation "EvolveOneBit" build initial state where there is only one qubit in state $| 1 \rangle$, and all others in state $| 0 \rangle$, evolves it for given number of steps, measures and print the result.
* Operation "EvolveSuperposition" build superposition of two initial states with exactly one qubit in state $| 1 \rangle$ (both with weights $\frac{1}{\sqrt{2}}$, evolves it for given number of states, measures and prints the result. 


In [1]:
open Microsoft.Quantum.Diagnostics;

operation LeftShift(qs: Qubit[]) : Unit {
    let N = Length(qs);
    for i in 0..N-2 {
        SWAP(qs[i], qs[i+1]);
    }
}

operation Rule90(qs: Qubit[]) : Unit {
    let N = Length(qs);
    for i in 3..N {
        CNOT(qs[N-i], qs[N-i+2]);
    }
    for i in 1..(N/2)-1 {
        CNOT(qs[2*i], qs[0]);
    }
    LeftShift(qs);    
}
operation MeasureAndPrint(qs: Qubit[]) : Unit {
    let N = Length(qs);
    mutable result = new Int[N];
    for i in 0..N-1 {
        if(M(qs[i]) == One) {
            set result w/= i <- 1;
        }
    }
    Message($"{result}");
}

operation EvolveOneBit(N: Int, pos: Int, steps: Int) : Unit {
    use qs = Qubit[N];
    X(qs[pos]);
    MeasureAndPrint(qs);
    for i in 1..steps {
        Rule90(qs);
        MeasureAndPrint(qs);
    }
    ResetAll(qs);
}

operation EvolveSuperposition(N: Int, pos1: Int, pos2: Int, steps: Int, attempts: Int) : Unit {
    use qs = Qubit[10];
        for attempt in 1..attempts {
        H(qs[pos1]);
        CNOT(qs[pos1], qs[pos2]);
        X(qs[pos2]);
        for i in 1..steps {
            Rule90(qs);
        }
        MeasureAndPrint(qs);
        ResetAll(qs);
    }
}

LeftShift, Rule90, MeasureAndPrint, EvolveOneBit, EvolveSuperposition

Let's simulate initial states with only 1 qubit set to $| 1 \rangle$:

In [2]:
%simulate EvolveOneBit N=10 pos=2 steps=10

[0,0,1,0,0,0,0,0,0,0]
[0,1,0,1,0,0,0,0,0,0]
[1,0,0,0,1,0,0,0,0,0]
[0,1,0,1,0,1,0,0,0,0]
[1,0,0,0,0,0,1,0,0,0]
[0,1,0,0,0,1,0,1,0,0]
[1,0,1,0,1,0,0,0,1,0]
[0,0,0,0,0,1,0,1,0,1]
[0,0,0,0,1,0,0,0,0,0]
[0,0,0,1,0,1,0,0,0,0]
[0,0,1,0,0,0,1,0,0,0]


()

In [3]:
%simulate EvolveOneBit N=16 pos=8 steps=10

[0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0]
[0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0]
[0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0]
[0,0,0,0,0,1,0,1,0,1,0,1,0,0,0,0]
[0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0]
[0,0,0,1,0,1,0,0,0,0,0,1,0,1,0,0]
[0,0,1,0,0,0,1,0,0,0,1,0,0,0,1,0]
[0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1]
[1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
[0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
[1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0]


()

As expected, we are getting the "Sierpinski triangle" pattern.

Now, let's simulate evolution of superposition. Here, we start with initial state
$\frac{1}{\sqrt{2}} |0000100000 \rangle + \frac{1}{\sqrt{2}} |0000010000 \rangle$.

In theory, we expect the result to be superposition of two basis states, each of which represents evolution of superposed initial states.

To confirm that, we simulate evolution of the same superposition 10 times, and each time measure the result.

In [4]:
%simulate EvolveSuperposition N=10 pos1=4 pos2=5 steps=5 attempts=10

[0,1,0,0,0,0,0,1,0,1]
[0,1,0,0,0,0,0,1,0,1]
[1,0,1,0,0,0,0,0,1,0]
[1,0,1,0,0,0,0,0,1,0]
[1,0,1,0,0,0,0,0,1,0]
[1,0,1,0,0,0,0,0,1,0]
[0,1,0,0,0,0,0,1,0,1]
[1,0,1,0,0,0,0,0,1,0]
[1,0,1,0,0,0,0,0,1,0]
[1,0,1,0,0,0,0,0,1,0]


()

As expected, we see only 2 different outcomes, and they appear with equal frequency. And we can check that those outcomes are exactly results of evolutions of states 0000100000 and 0000010000.

Closing remarks:

* This notebook was written for the [Q# Advent Calendar 2021](https://devblogs.microsoft.com/qsharp/q-advent-calendar-2021/). If you are interested in Quantum Computing or in the Q# programming language, you should go check it out.

* This notebook is part of my project "Quantum Circuits for Elementary Cellular Automata". In that project I identified all 22 (of 256) rules which can be implemented on a quantum computer, and built cirxuits for all of them. Please check out my paper [here](http://dx.doi.org/10.13140/RG.2.2.22346.08641) and code [here](https://github.com/fedimser/quant_comp/tree/master/cellular_automata). 

* This is a Jupyter Notebook. You can download and execute it. For that, in addition to Python and Jupyter, you need to install Q# kernel following [these instructions](https://docs.microsoft.com/en-us/azure/quantum/install-command-line-qdk#jupyter-notebooks-using-the-net-cli). 